<style>
* {
    font-family: consolas,'Microsoft YaHei';
}
p {
    line-height: 1.5em;
    font-size: 20px;
}
.gap {
    display: inline-block;
    padding: 0 10px;
}
</style>
---

<center>
<h1>第九课</h1>
</center>

## **主题**：代理的使用

## 纲要
### 一. 代理的基本设置
### 二. 代理池的维护
### 三. 使用代理爬取微信公众号文章
---
 

<center><h2>一. 代理的基本设置</h2></center>

<p>前面课程稍微提过，使用代理的目的有两个：其一是解决地理区域联网受限，其二是为进行IP 伪装。对于IP 伪装的应用场景，即为避免Web服务器对网络爬虫的封锁。常见的代理服务有HTTP 代理服务 和 SOCKS 代理服务两种类型。有条件的话我们可以自己搭建一台代理服务器，便捷的方式是使用第三方免费或付费的代理服务。</p>

<center><img src="../../image/unnamed.png" /></center>
<p>
下面通过实例来分别介绍HTTP代理服务和 SOCKS 代理服务的配置。
</p>


In [2]:
from urllib.error import URLError
from urllib.request import ProxyHandler, build_opener

proxy = 'username:password@127.0.0.1:9743'
proxy_handler = ProxyHandler({
    'http': 'http://' + proxy,
    'https': 'https://' + proxy
})
opener = build_opener(proxy_handler)
try:
    response = opener.open('http://httpbin.org/get')
    print(response.read().decode('utf-8'))
except URLError as e:
    print(e.reason)


[WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond


In [ ]:
# 首先需要在命令行安装PySocks: pip3 install PySocks

import socks
import socket
from urllib import request
from urllib.error import URLError

socks.set_default_proxy(socks.SOCKS5, '127.0.0.1', 9742)
socket.socket = socks.socksocket
try:
    response = request.urlopen('http://httpbin.org/get')
    print(response.read().decode('utf-8'))
except URLError as e:
    print(e.reason)

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# proxy = 'username:password@127.0.0.1:9743'
proxy = '127.0.0.1:8888'

def browserwraper(*args):
    opts = Options()
    # opts.add_argument('--headless')
    opts.add_argument('--disable-gpu')
    for arg in args:
        opts.add_argument(arg)
    return webdriver.Chrome(executable_path=r'D:\ENV\chromedriver.exe', options=opts)


browser = browserwraper('--proxy-server=http://{}'.format(proxy))
browser.get('http://httpbin.org/get')

In [ ]:
# 带有认证的代理

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import zipfile

ip = '127.0.0.1'
port = 9743
username = 'foo'
password = 'bar'

manifest_json = """
{
    "version": "1.0.0",
    "manifest_version": 2,
    "name": "Chrome Proxy",
    "permissions": [
        "proxy",
        "tabs",
        "unlimitedStorage",
        "storage",
        "<all_urls>",
        "webRequest",
        "webRequestBlocking"
    ],
    "background": {
        "scripts": ["background.js"]
    }
}
"""

background_js = """
var config = {
        mode: "fixed_servers",
        rules: {
          singleProxy: {
            scheme: "http",
            host: "%(ip)s",
            port: %(port)s
          }
        }
      }

chrome.proxy.settings.set({value: config, scope: "regular"}, function() {});

function callbackFn(details) {
    return {
        authCredentials: {
            username: "%(username)s",
            password: "%(password)s"
        }
    }
}

chrome.webRequest.onAuthRequired.addListener(
            callbackFn,
            {urls: ["<all_urls>"]},
            ['blocking']
)
""" % {'ip': ip, 'port': port, 'username': username, 'password': password}

plugin_file = 'proxy_auth_plugin.zip'
with zipfile.ZipFile(plugin_file, 'w') as zp:
    zp.writestr("manifest.json", manifest_json)
    zp.writestr("background.js", background_js)
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_extension(plugin_file)
browser = webdriver.Chrome(chrome_options=chrome_options)
browser.get('http://httpbin.org/get')

<center><h2>二. 代理池的维护</h2></center>

<p>当我们使用代理服务时，无论它是免费的还是收费的都会面临一个问题，即代理服务器也会因频繁访问目标站点而被封禁，或者代理服务器突然出故障或网络繁忙。这时，我们需要自己维护一份保存运行良好的代理服务器信息。好比这里有一个按顺序排列的队列池，排序按每台服务器响应速度所拟的一个权重得分，使最佳者排在队列最上或最下面。这样每次使用代理时都随机得到响应速度达到指定阀值的代理服务器。注意，这里不能每次都选择最佳的那台服务器。因为那样可能会每次得到同一台代理服务器，同样会造成被目标站点封禁。</p>

<p>
以上是代理池的基本实现思路，出于技术层面选型考虑，可以使用redis的sort set类型数据来动态处理这些服务器信息。sort set 称为有序集合，除了支持键值外，还可赋予一个作为权重的score值。每添加一个新的值，redis会根据权重自动排序。
</p>

<center><img src="../../image/zset.png" /></center>

由于代理池的维护并不复杂，这里就不作介绍。而直接通过github上一个开源的代理池项目来介绍代理池。

<p>这里要介绍的代理池是<a href="https://github.com/jhao104/proxy_pool">proxy_pool</a>.它是类似的Python程序库中用的人较多的一个，具体使用流程详见项目首页Readme文档。</p>

<center><h2>三. 使用代理爬取微信公众号文章</h2></center>
<p>（作为实战作业）在搜狗微信网站搜索“马拉多纳”，获取前3页内容中的每条新闻的标题、公众号名称以及发布日期，结果保存到Mysql。/p>